In [ ]:
# Install necessary dependencies
!pip install nest_asyncio diffusers[torch] torch torchvision torchaudio pyngrok fastapi uvicorn pydantic jinja2

In [ ]:
!ngrok config add-authtoken 2obzVQ2rFZANwZD8Z3iiQmcGixW_4gj8SRP8RS8q3zNBTLTtk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok authtoken 2obzVQ2rFZANwZD8Z3iiQmcGixW_4gj8SRP8RS8q3zNBTLTtk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Import necessary libraries
import nest_asyncio
from diffusers import StableDiffusionPipeline
import torch
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import base64
from io import BytesIO
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse

# Apply nest_asyncio to allow running asyncio event loop within a Jupyter/Colab environment
nest_asyncio.apply()

# Load the Stable Diffusion model
pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipeline.to("cuda")  # Move the model to GPU

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.30.3",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
# Create a FastAPI app
app = FastAPI()

# Set up Jinja2 template engine
templates = Jinja2Templates(directory="templates")

# Define a request model for prompt input
class PromptRequest(BaseModel):
    prompt: str

# Serve the HTML form for entering prompts
@app.get("/", response_class=HTMLResponse)
async def get_form(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

# Define the endpoint for generating images
@app.post("/generate/")
async def generate_image_api(request: PromptRequest):
    try:
        # Generate the image from the prompt
        image = pipeline(request.prompt).images[0]

        # Convert the image to base64 to send it as a response
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        # Return the image as base64 in the response
        return {"message": "Image generated successfully", "image": img_str}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Start the ngrok tunnel to expose the API
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI server in the Colab environment
# We don't need to use 'app:app' format here, just pass the app object directly.
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [2872]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://5a7b-34-143-173-83.ngrok-free.app" -> "http://localhost:8000"
INFO:     196.136.13.24:0 - "GET / HTTP/1.1" 200 OK
INFO:     196.136.13.24:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     196.136.13.24:0 - "POST /generate/ HTTP/1.1" 200 OK


Token indices sequence length is longer than the specified maximum sequence length for this model (83 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', perfect for outdoor adventures.']


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     196.136.13.24:0 - "POST /generate/ HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     196.136.13.24:0 - "POST /generate/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2872]


In [ ]:
ngrok.disconnect(public_url)